## Imports:

In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn import preprocessing

import spacy
from spacy.lang.en import English
from spacy import displacy
nlp = spacy.load('en')
from IPython.display import HTML
import logging
logging.getLogger('tensorflow').disabled = True #OPTIONAL - to disable outputs from Tensorflow

ModuleNotFoundError: No module named 'spacy'

## Create sentence embeddings from Elmo via TensorFlow Hub
https://tfhub.dev/google/elmo/1

In [5]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

## Read Plot Sentences

In [6]:
# Change Directory to Data
# read from data
# Create elmoWeights Folder, in not exists
# Write weights to data/emloWeights

import os
rootdir = '/Users/jasonboada/Desktop/cis/'

# elmodir = os.path.join(datadir, 'elmoWeightsAll') 
elmodir = rootdir+'elmoWeightsAll'

# datadir = os.path.join(rootdir, 'data') 
datadir = '/Users/jasonboada/Desktop/cis/scrgt-bdso-cc-datascience/datapipeline/data/person/themoviedb/'

os.chdir(datadir)

# create elmoWeights directory - if it does not exist
if not os.path.exists(elmodir):
    os.makedirs(elmodir) 

## READ PLOT DATA - ALL from PKL file

In [10]:
import json
bio_dict = {}
for f in os.listdir(datadir):
    data = json.load(open(datadir+f))
    bio = data["biography"]
    bio_dict[data["id"]] = bio

In [63]:
      
sentencesAll = list(bio_dict.values())

In [ ]:
sentencesAll

In [18]:
def cleanSentences(sentences):
    punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'  #TBD
        #TBD - find sentence splitter - 
        # remove punctuation, strip white space; remove non-standard characters; remove numbers; 
    i = 0
    for s in sentences:
        s = s.strip()
        s = s.lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ').replace("[0-9]", " ")
        s = s.replace(',', '').replace('.','') #TBD - replace with punctuation
        s = s[0:280]
        sentences[i] = s
        i += 1
    return sentences

In [19]:
# FOR Data Frame Inputs
# sentencesDF = moviePlots.values.tolist()
# sentencesDF = plotDF['Plot'].tolist()
# sentences = cleanSentences(sentencesDF)

# For Pickle Input
sentences = cleanSentences(sentencesAll)

In [20]:
# Test - 
sentences[99]
# len(sentences)

"a welsh-born english actor in addition to starring roles in big budget hollywood films he has long been heavily involved in films produced by independent producers and art houses bale first caught the public eye when he was cast in the starring role of steven spielberg's empire o"

# Score Sentences in Batches

In [21]:
# score n Sentences at time - to pipeline this
def defineEmbeddings (start, end):    
    embeddings = embed(
        sentences[start:end],
        signature="default",
        as_dict=True)["default"]
    return embeddings

In [22]:
# score sentences
def scoreSentences (embeddings):
    %%time

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        x = sess.run(embeddings)
    return x

In [23]:
# Save Weights - use names based on Movie 
import pickle as pkl

def saveWeights(elmoFN, x):
    os.chdir(elmodir)
    with open(elmoFN,'wb') as f: pkl.dump(x, f)
    # TBD - Close pkl?
    return

## Score N sentences at a time
## DO NOT EXECUTE unless we change the sentences  (15,316)

In [24]:
import math 

n = 500                      # Specify number of sentences per file  (was 500)
numPlots = len(sentences)
numfiles = math.ceil(numPlots/n) 

start = 0
end = n
for file in range(1,numfiles+1):
    elmoFN = 'elmoIMDBweights' + str(start) + 'to' + str(end) + '.pkl'
    print('Score sentences:', elmoFN)

        # Define Embeddings, Score, and Store Weights one Batch of N at time
    embeddings = defineEmbeddings(start, end)
    x = scoreSentences(embeddings)
    saveWeights(elmoFN,x)
    start = end
    end = end+n

Score sentences: elmoIMDBweights0to500.pkl
CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.11 µs


## Read and Concatenate all weight arrays in elmoWeights to build

TBD limit to a segment (e.g., Drama)


In [35]:
# Read Weights - use names based on Movie 
# TBD - add row num -- TBD Save with Movie ID 
# TBD - missing a row ERROR
import pickle as pkl
import numpy as np
import glob

def readElmoWeightFile(file):
    with open(file, 'rb') as f:
        elmo_file = pkl.load(f)
    return list(elmo_file)
  
# xin = readWeights(elmodir) # Removed this approach - Need to read weights in order
def readWeights(elmodir):
    
    xin = readElmoWeightFile('elmoIMDBweights0to500.pkl')

    return xin

In [36]:
pwd

'/Users/jasonboada/Desktop/cis/elmoWeightsAll'

In [37]:
os.chdir(elmodir)
xin = readWeights(elmodir)
len(xin) # 15316

100

In [1]:
import pickle
import json

bio_dict = pickle.load( open( './models/bio_dict.pkl', 'rb' ) )
xin = pickle.load( open( './models/elmoIMDBweights0to500.pkl', 'rb' ) )


In [ ]:
bio_dict

In [5]:
celebrities =pd.read_csv('./models/celebrityLookup.csv' , names=['Rank', 'Actor'], dtype=str)
celebrities.head()

,Rank,Actor
0,1,Robert Downey Jr.
1,2,Samuel L. Jackson
2,3,Scarlett Johansson
3,4,Tom Hanks
4,5,Bradley Cooper


In [6]:
celebrityLookup={}
for i, row in celebrities.iterrows():
    celebrityLookup.update({ row['Rank'] : row['Actor']})

celebrityLookup['10']

'Harrison Ford'

In [7]:
len(xin[1])

1024

In [14]:
import requests
from pprint import pprint
from datetime import datetime
from elasticsearch import Elasticsearch
import json


ESHOST='localhost'
ESPORT='9200'
#https://elasticsearch-dev.ocp.bdso.salientcrgt-bdso.com/
# ESHOST ='elasticsearch-dev.ocp.bdso.salientcrgt-bdso.com'
# ESPORT='80'
es= Elasticsearch([{'host':ESHOST,'port':ESPORT}])
# es=Elasticsearch(['http://localhost:9200/'])
_index='bios'
esIndex = 'http://'+ESHOST+':'+ESPORT+'/'+_index    
_doc= 'bio'

headers={"Content-Type": "application/json"}

In [15]:
request_body = {
  "mappings": {
    "properties": {
      "_actorID" : {
        "type" : "keyword"
      },  
      "celebrity" : {
        "type" : "keyword"
      },         
      "bio_vector": {
        "type": "dense_vector",
        "dims": len(xin[0])
      }

    }
  }
}

try:
    es.indices.delete(index = _index)
except:
    pass
    
es.indices.create(index = _index, body = request_body)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'bios'}

In [16]:
ids = list(bio_dict.keys())
for i, v in zip(ids, xin):
    _id = str(i)
    #print(_id)
    doc = {
        #_actorID	actor	age	birthdate	gender	height
        "_actorID" : i,
        "celebrity" : celebrityLookup[_id],
        "bio_vector": [float(x) for x in v], 
        }
    r = requests.put(esIndex+'/_doc/'+_id, headers= headers, data = json.dumps(doc))
    print (r.text)

{"_index":"bios","_type":"_doc","_id":"49","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":0,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"6","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":1,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"73","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":2,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"24","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":3,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"32","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":4,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"65","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":5,"_primary_term":1}
{"_index":"bios","_type":"_doc","_id":"12","_version":1,"result":"created","_

In [17]:
r = requests.get(esIndex+'/_doc/1')
pprint(r.text)

('{"_index":"bios","_type":"_doc","_id":"1","_version":1,"_seq_no":43,"_primary_term":1,"found":true,"_source":{"_actorID": '
 '"1", "celebrity": "Robert Downey Jr.", "bio_vector": [0.04837771877646446, '
 '0.022511877119541168, -0.10536447167396545, -0.1552850306034088, '
 '0.2935081422328949, 0.3959132432937622, -0.16279493272304535, '
 '0.32249748706817627, 0.23131726682186127, 0.07074576616287231, '
 '0.013724659569561481, -0.018507424741983414, -0.25354212522506714, '
 '-0.15150010585784912, -0.2948114573955536, 0.044774092733860016, '
 '0.2890183627605438, 0.08981198072433472, -0.10185448825359344, '
 '0.003111480502411723, 0.14141295850276947, 0.225153848528862, '
 '0.03843172639608383, -0.08233527094125748, 0.45199763774871826, '
 '-0.15272027254104614, 0.06622729450464249, 0.19484412670135498, '
 '0.29314160346984863, 0.565075695514679, 0.3793236017227173, '
 '-0.18968883156776428, 0.2955780327320099, -0.07489267736673355, '
 '-0.017330626025795937, 0.05929869785904884, -0.260

In [20]:
vec = json.loads(r.text)["_source"]["bio_vector"]
q={
  "_source": {
        "includes": [ "name"]
        },
  "query": {
    "script_score": {

      "query" : {
        "match_all" : {}
      },
      "script": {
        "source": "cosineSimilarity(params.query_vector, doc['bio_vector']) + 1.0", 
        "params": {
          "query_vector": vec
        }
      }
    }
  }
}
res= es.search(index=_index, body=q)

pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '1',
                    '_index': 'bios',
                    '_score': 2.0,
                    '_source': {},
                    '_type': '_doc'},
                   {'_id': '80',
                    '_index': 'bios',
                    '_score': 1.8732605,
                    '_source': {},
                    '_type': '_doc'},
                   {'_id': '12',
                    '_index': 'bios',
                    '_score': 1.862133,
                    '_source': {},
                    '_type': '_doc'},
                   {'_id': '30',
                    '_index': 'bios',
                    '_score': 1.8555069,
                    '_source': {},
                    '_type': '_doc'},
                   {'_id': '33',
                    '_index': 'bios',
                    '_score': 1.8527236,
                    '_source': {},
                    '_type': '_doc'},
             

In [62]:
[hit['_source']['_actorID'] for hit in res['hits']['hits']]

[236695, 60898, 1136406, 10990, 139820, 7060, 1315036, 84214, 10980, 10989]

# REDUCE DIMENSIONS

### Visualize the sentence arrays in reduced Dimensions using PCA and TSNE

Use PCA and t-SNE to reduce the 1,024 dimensions which are output from ELMo down to 2 so that we can review the outputs from the model.

In machine learning, dimensionality reduction is the process of reducing the number of random variables under consideration by obtaining a set of principal variables. By reducing the dimension of your feature space, you have fewer relationships between features to consider which can be explored and visualized easily and also you are less likely to overfit your model. https://www.datacamp.com/community/tutorials/introduction-t-sne

#### Principal Component Analysis or PCA 
Principal Component Analysis (PCA)is a linear feature extraction technique. It performs a linear mapping of the data to a lower-dimensional space in such a way that the variance of the data in the low-dimensional representation is maximized. It does so by calculating the eigenvectors from the covariance matrix. The eigenvectors that correspond to the largest eigenvalues (the principal components) are used to reconstruct a significant fraction of the variance of the original data.

In simpler terms, PCA combines your input features in a specific way that you can drop the least important feature while still retaining the most valuable parts of all of the features. As an added benefit, each of the new features or components created after PCA are all independent of one another.
t-Distributed Stochastic Neighbor Embedding (t-SNE)

#### t-Distributed Stochastic Neighbor Embedding (t-SNE) 
is a non-linear technique for dimensionality reduction that is particularly well suited for the visualization of high-dimensional datasets. It is extensively applied in image processing, NLP, genomic data and speech processing.


In [ ]:
### NOTE - this takes a bit of time to reduce 1024 by 15316 array into 2 dimensions
from sklearn.decomposition import PCA

pca = PCA(n_components=50)  # was 50
y = pca.fit_transform(xin)

# t-Distributed Stochastic Neighbor Embedding (t-SNE) - Dimensionality Reduction technique
from sklearn.manifold import TSNE
y = TSNE(n_components=2).fit_transform(y)

In [ ]:
# Build a unified data frame (udf) with columns: MovieID, Plot, Genre, and Y-transform
# udf = pd.DataFrame(columns=['MovieID', 'Plot', 'Genre','Y-Transform'])

i=0
row_list = []
for key in plot_dict: 
    row_dict = {'MovieID':key,
                'Plot':sentences[i],
                'Genre':genre_dict[key],
                'Y-Transform-X': y[i][0],
                'Y-Transform-Y': y[i][1],
               }
    row_list.append(row_dict)
    i += 1
udf = pd.DataFrame(row_list)

In [ ]:
udf.to_csv("UnifiedData.csv", index=False)

In [ ]:
# Test below

In [ ]:
udf.head()

In [ ]:
udf = pd.read_csv("../data/elmoWeightsAll/UnifiedData.csv", dtype={'MovieID':str} )
udf.head()